In [12]:
import importlib
import thumbnails
import db_connect
import photo_export
import path_management
from datetime import date
from tqdm import tqdm
from osxphotos import PhotoInfo

importlib.reload(thumbnails)
importlib.reload(db_connect)
importlib.reload(photo_export)
importlib.reload(path_management)

from thumbnails import ThumbnailDef, Thumbnailer
from db_connect import DbConnector
from photo_export import PhotoExporter
from path_management import PathManager

In [13]:
small = ThumbnailDef(120, 'small')
medium = ThumbnailDef(400, 'medium')
large = ThumbnailDef(1600, 'large')

sizes = [small, large, medium] 
thumbnailer = Thumbnailer(sizes)

In [14]:
from_date = date(2011, 8, 1)
to_date = date(2011, 9, 1)
paths = PathManager()
test_connector = DbConnector('./test.postgres_config', paths, thumbnailer.name_modifiers)
prod_connector = DbConnector('./prod.postgres_config', paths, thumbnailer.name_modifiers)
exporter = PhotoExporter(paths)

games = prod_connector.get_games(from_date, to_date)

In [16]:
for game in tqdm(games):
    exporter.export_photos_for_game(game)

 25%|██▌       | 2/8 [00:00<00:00, 13.75it/s]

skipping missing photo A843C736-6C6A-4793-ADD7-43F40AACA258.JPG
skipping missing photo 74D652BA-22F9-4ECC-89EC-2CF32C060EAB.JPG
skipping missing photo A81B0891-97BA-4ADA-B347-EA9E69FAC50E.JPG
skipping missing photo A390A4B8-E325-437F-8C7A-9DD39EC5E257.JPG


100%|██████████| 8/8 [00:01<00:00,  5.37it/s]

skipping missing photo 94D8E58D-62E3-4565-8ADE-BF7AC13BED48.JPG


At this point, review the photos in the "preview" directories and remove any that shouldn't be uploaded. Removing it from the preview directory is sufficient.

In [17]:
to_thumbnail = []
for game in games:
    to_thumbnail.extend(exporter.get_photos_to_thumbnail(game))

In [20]:
thumbnailer.thumbnail_many(to_thumbnail)

In [21]:
to_upload: list[PhotoInfo] = []
for game in games:
    to_upload.extend(exporter.get_photos_to_upload(game))

AttributeError: 'PhotoExporter' object has no attribute 'get_photos_to_upload'

In [ ]:
# TODO:
# - implement get_photos_to_upload
# - implement upload/insert process
#    - check if file is in bucket
#       - if not, upload it
#    - check if resource is in db
#       - if not, insert it
#    - insert any files into db that aren't already there
test_connector.import_resources(1, to_upload)